In [1]:
import pythonradex

# # import necessary modules
from pythonradex import radiative_transfer, helpers, LAMDA_file, atomic_transition
from scipy import constants
import numpy as np
import matplotlib.pyplot as plt

from scipy.stats import norm

In [2]:
# pyhonradex needs a file containing the atomic data, import it, this is a specific file for CO from LAMBDA database
datafilepath = r"C:\Users\alios\OneDrive - University College London\Desktop\UCL\Year3\group project\codes\co_datafile.dat"  # file downloaded from LAMDA database


# define the geometry of the nebula
geometry = "uniform sphere"

# define the parameters of the nebula
line_profile_type = "Gaussian"  # line profile, can be "Gaussian",  “rectangular”,"LVG sphere", "LVG slab”
width_v = 5 * constants.kilo  # line width in m/s, costants.kilo is 1 km/s

cloud = radiative_transfer.Cloud(
    datafilepath=datafilepath,
    geometry=geometry,
    line_profile_type=line_profile_type,
    width_v=width_v,
)

N = 1e18 / constants.centi**2  # CO column density in m-2
Tkin = 30  # kinetic temperature in [K]

# collider densities in cm-3:
para_h2_density = 1e8
ortho_h2_density = 3e8

# collider densities in m-3 (phytonradex takes the densities in m-3):
collider_densities_LTE = {
    "para-H2": para_h2_density / constants.centi**3,
    "ortho-H2": ortho_h2_density / constants.centi**3,
}

# define the background radiation field, in this case we assume that the background is zero, z is the redshift
ext_background = helpers.generate_CMB_background(z=0)

# no dust:
T_dust = 0
tau_dust = 0

# use high colliders density to simulate LTE
cloud.update_parameters(
    N=N,
    Tkin=Tkin,
    collider_densities=collider_densities_LTE,
    ext_background=ext_background,
    T_dust=T_dust,
    tau_dust=tau_dust,
)

# solve the radiative trasfer equation, i.e. calculate the level population with an iterative method
cloud.solve_radiative_transfer()

In [3]:
# store the data
data_co = LAMDA_file.read(datafilepath, read_frequencies=True)

# Initialize g_weigth
levels = data_co["levels"]
rad_transitions = data_co["radiative transitions"]
coll_transitions = data_co["collisional transitions"]
quantum_numbers = data_co["quantum numbers"]

g_weigth = np.array([level.g for level in levels])
g_up = np.array([gup.name for gup in rad_transitions])

In [4]:
number_level = np.array([level.number for level in levels])

In [5]:
Eu = np.array([level.E for level in levels]) / constants.k

Eu_test = np.array([level.up.E for level in rad_transitions]) / constants.k

In [6]:
Eu

array([   0.        ,    5.53214517,   16.59617609,   33.19188137,
         55.31833031,   82.974736  ,  116.16031175,  154.87340764,
        199.11280441,  248.87685379,  304.16332873,  364.97043585,
        431.29580629,  503.13678297,  580.49056589,  663.35449669,
        751.72520052,  845.599158  ,  944.97313597, 1049.84303913,
       1160.20491752, 1276.0546769 , 1397.3877882 , 1524.19972692,
       1656.48539072, 1794.2399659 , 1937.45806361, 2086.1342947 ,
       2240.26298185, 2399.83801775, 2564.8534387 , 2735.3029919 ,
       2911.18013782, 3092.47804981, 3279.18990371, 3471.30844029,
       3668.82625666, 3871.73609478, 4080.02983772, 4293.69964515,
       4512.73740391])

In [7]:
Eu_test

array([   5.53214517,   16.59617609,   33.19188137,   55.31833031,
         82.974736  ,  116.16031175,  154.87340764,  199.11280441,
        248.87685379,  304.16332873,  364.97043585,  431.29580629,
        503.13678297,  580.49056589,  663.35449669,  751.72520052,
        845.599158  ,  944.97313597, 1049.84303913, 1160.20491752,
       1276.0546769 , 1397.3877882 , 1524.19972692, 1656.48539072,
       1794.2399659 , 1937.45806361, 2086.1342947 , 2240.26298185,
       2399.83801775, 2564.8534387 , 2735.3029919 , 2911.18013782,
       3092.47804981, 3279.18990371, 3471.30844029, 3668.82625666,
       3871.73609478, 4080.02983772, 4293.69964515, 4512.73740391])

In [8]:
for i, tran in enumerate(rad_transitions):
    gu = tran.up.g
    Eu = tran.up.E
    print(f"Transition {i}:")
    print("Statistical weight of upper level ", gu)
    print("Energy of upper level Eu", Eu / constants.k)
    print()

print("statsitical weight of upper level ", gu)
print("Energy of upper level Eu", Eu / constants.k)

Transition 0:
Statistical weight of upper level  3.0
Energy of upper level Eu 5.532145167854433

Transition 1:
Statistical weight of upper level  5.0
Energy of upper level Eu 16.596176090653508

Transition 2:
Statistical weight of upper level  7.0
Energy of upper level Eu 33.19188137466572

Transition 3:
Statistical weight of upper level  9.0
Energy of upper level Eu 55.31833030822088

Transition 4:
Statistical weight of upper level  11.0
Energy of upper level Eu 82.97473600266304

Transition 5:
Statistical weight of upper level  13.0
Energy of upper level Eu 116.16031175493842

Transition 6:
Statistical weight of upper level  15.0
Energy of upper level Eu 154.8734076419077

Transition 7:
Statistical weight of upper level  17.0
Energy of upper level Eu 199.11280441095295

Transition 8:
Statistical weight of upper level  19.0
Energy of upper level Eu 248.8768537863343

Transition 9:
Statistical weight of upper level  21.0
Energy of upper level Eu 304.1633287284865

Transition 10:
Statis

In [9]:
g_up[0]

np.str_('1-0')

In [10]:
def extract_upper_number(np_str):
    return int(np_str.split("-")[0])


# Example usage
first_number = extract_upper_number(g_up[0])

print(first_number)  # Output: 1

1


In [11]:
g_weigth

array([ 1.,  3.,  5.,  7.,  9., 11., 13., 15., 17., 19., 21., 23., 25.,
       27., 29., 31., 33., 35., 37., 39., 41., 43., 45., 47., 49., 51.,
       53., 55., 57., 59., 61., 63., 65., 67., 69., 71., 73., 75., 77.,
       79., 81.])

# Testing methanol 

In [12]:
# pyhonradex needs a file containing the atomic data, import it, this is a specific file for CO from LAMBDA database
datafilepath = (
    r"C:\Users\alios\OneDrive - University College London\Desktop\UCL\Year3\group project\codes\e_ch3oh.dat"  # file downloaded from LAMDA database
)


# define the geometry of the nebula
geometry = "uniform sphere"

# define the parameters of the nebula
line_profile_type = "Gaussian"  # line profile, can be "Gaussian",  “rectangular”,"LVG sphere", "LVG slab”
width_v = 5 * constants.kilo  # line width in m/s, costants.kilo is 1 km/s

cloud = radiative_transfer.Cloud(
    datafilepath=datafilepath,
    geometry=geometry,
    line_profile_type=line_profile_type,
    width_v=width_v,
)

N = 1e18 / constants.centi**2  # CO column density in m-2
Tkin = 30  # kinetic temperature in [K]

# collider densities in cm-3:
para_h2_density = 1e8
ortho_h2_density = 3e8

# collider densities in m-3 (phytonradex takes the densities in m-3):
LTE_collider_densities_h2only = {"H2": 1e8 / constants.centi**3}

# define the background radiation field, in this case we assume that the background is zero, z is the redshift
ext_background = helpers.generate_CMB_background(z=0)

# no dust:
T_dust = 0
tau_dust = 0

# use high colliders density to simulate LTE
cloud.update_parameters(
    N=N,
    Tkin=Tkin,
    collider_densities=LTE_collider_densities_h2only,
    ext_background=ext_background,
    T_dust=T_dust,
    tau_dust=tau_dust,
)

# solve the radiative trasfer equation, i.e. calculate the level population with an iterative method
cloud.solve_radiative_transfer()

79-78: tau_nu0 = -0.0215
121-120: tau_nu0 = -0.0107
137-136: tau_nu0 = -4.31e-06


c:\Users\alios\AppData\Local\Programs\Python\Python312\Lib\site-packages\pythonradex\radiative_transfer.py:110: UserWarning: some lines are overlapping, but treatement of overlapping lines not activated
  warnings.warn('some lines are overlapping, but treatement of'
c:\Users\alios\AppData\Local\Programs\Python\Python312\Lib\site-packages\pythonradex\radiative_transfer.py:333: UserWarning: negative optical depth!
  warnings.warn('negative optical depth!')


In [21]:
# store the data
data = LAMDA_file.read(datafilepath, read_frequencies=False, read_quantum_numbers=True)

# Initialize g_weigth
levels = data["levels"]
rad_transitions = data["radiative transitions"]
coll_transitions = data["collisional transitions"]
quantum_numbers = data["quantum numbers"]


In [ ]:
# selected_quantum_numbers_with_indices = [(i, qn) for i, qn in enumerate(quantum_numbers) if qn.split("_")[1] == "1"]
# indices = [i for i, qn in selected_quantum_numbers_with_indices]
# print(indices)

[6, 8, 12, 17, 22, 27, 36, 43, 53, 64, 77, 91, 104, 120, 135]


In [ ]:
# class MyClass:
#     selected_quantum_numbers_with_indices = [(i, qn) for i, qn in enumerate(quantum_numbers) if qn.split("_")[1] == "1"]
#     indices = [i for i, qn in selected_quantum_numbers_with_indices]
#     print(indices)

#     def __init__(self, cloud_name, data_of_molecule, column_density_species, width_v, Tkin):
#         """
#         Initialize the class with the cloud object, the data of the molecule, the column density of the species, the width of the line and the kinetic temperature.

#         Parameters
#         ----------
#         cloud_name : pythonradex.Cloud
#              The cloud object.
#         data_of_molecule : dict
#              The data of the molecule.
#         column_density_species : float
#              The column density of the species in m^2.
#         width_v : float
#              The width of the line in m/s
#         Tkin : float
#              The kinetic temperature in [K].
#         """
#         self.cloud_name = cloud_name
#         self.data_of_molecule = data_of_molecule
#         self.column_density_species = column_density_species
#         self.width_v = width_v
#         self.Tkin = Tkin
#         self.extract_and_calculate()  # this allows optical_depth to use the extracted values from the extract_and_calculate method

#     def get_transition_populations(self, names_transitions):
#         """
#         Relates all the transitions in names_transitions with their corresponding population densities.

#         Parameters
#         ----------
#         names_transitions: (list of str):
#              List of transitions in the format ['79-78', '14-13', ...], where each element is a string with the format "upper_level-lower_level".

#         Returns
#         -------
#              tuple: Two arrays, one for upper and one for lower populations density.
#         """
#         upper_populations = []
#         lower_populations = []
#         for transition in names_transitions:
#             upper_level, lower_level = map(int, transition.split("-"))  # split the string and convert to integers
#             upper_pop = self.cloud_name.level_pop[upper_level]
#             lower_pop = self.cloud_name.level_pop[lower_level]
#             upper_populations.append(upper_pop)
#             lower_populations.append(lower_pop)

#         nu = np.array(upper_populations)
#         nl = np.array(lower_populations)
#         # compute upper level population density, given by: fractional population density x total column density
#         Nu = nu * self.column_density_species
#         Nl = nl * self.column_density_species
#         return Nu, Nl

#     def extract_and_calculate(self, debug=False):
#         """
#         Extract values from the cloud object and the data file, and calculate gamma factor

#         Parameters
#         ----------
#         debug : boolean, optional
#              If True, print debugging information. Default is False.

#         Returns
#         --------
#         Tex: np.array
#              The excitation temperature of the molecule.
#         tau: np.array
#              The optical depth of the transitions.
#         Aul : np.array
#              The Einstein coefficient for spontaneous emission [s^-1].
#         Bul : np.array
#              The Einstein coefficient for stimulated emission in [sr m2 Hz / Jy].
#         nu0_array : np.array
#              The rest frequency of the transition in Hz.
#         Eu : np.array
#              The energy of the upper level in [K].
#         gu : np.array
#              The statistical weight of the upper level.
#         gamma: np.array
#              Gamma factors, which is calculated using: gamma = (8 * np.pi * k * nu0**2) / (h * c**3 * Aul)
#         Nu: np.array
#              Upper level population density in [m-2].
#         Nl: np.array
#              Lower level population density in [m-2].
#         FWHM_each_transition: np.array
#              Frequency width of each transition in [Hz].
#         """
#         # Extract values
#         self.Tex = self.cloud_name.Tex
#         self.Aul = self.cloud_name.emitting_molecule.A21
#         self.Bul = self.cloud_name.emitting_molecule.B21
#         self.nu0_array = self.cloud_name.emitting_molecule.nu0
#         self.tau = self.cloud_name.tau_nu(self.nu0_array)

#         # levels = self.data_of_molecule["levels"]
#         rad_transitions = self.data_of_molecule["radiative transitions"]

#         # Extract upper energy levels in [J] and convert to Kelvin
#         self.Eu = np.array([transition.up.E for transition in rad_transitions]) / constants.k

#         # Get the statistical weight of the upper and lower levels
#         self.gu = np.array([transition.up.g for transition in rad_transitions])
#         self.glow = np.array([transition.low.g for transition in rad_transitions])

#         # Get names of transitions
#         names_transitions = np.array([tran.name for tran in rad_transitions])

#         # Compute upper and lower level population densities
#         self.Nu, self.Nl = self.get_transition_populations(names_transitions)

#         # Convert velocity width to frequency width
#         self.FWHM_each_transition = self.nu0_array * (self.width_v / constants.c)

#         # Calculate gamma factor
#         self.gamma = (8 * np.pi * constants.k * self.nu0_array**2) / (constants.h * constants.c**3 * self.Aul)

#         if debug:
#             # Debugging print statements
#             print("Size of nu0_array:", len(self.nu0_array))
#             print("Size of Eu:", len(self.Eu))
#             print("E upper levels:", self.Eu)
#             print("Size of gu:", len(self.gu))
#             print("size exitation temperature Tex:", len(self.Tex))
#             print(" gu:", self.gu)
#             print("Size of glow:", len(self.glow))
#             print(" glow:", self.glow)
#             print("size Aul:", len(self.Aul))
#             print("size FWHM_each_transition:", len(self.FWHM_each_transition))
#             print("size gamma:", len(self.gamma))
#             print("size Nu:", len(self.Nu))
#             print("size Nl:", len(self.Nl))

#         return (
#             self.Tex,
#             self.tau,
#             self.Aul,
#             self.Bul,
#             self.nu0_array,
#             self.Eu,
#             self.gu,
#             self.glow,
#             self.gamma,
#             self.Nu,
#             self.Nl,
#             self.FWHM_each_transition,
#         )

#     def get_selected_variables(self):
#         """
#         Returns the variables in extract_and_calculate which have indices selected by selected_quantum_numbers_with_indices.

#         Returns
#         -------
#         dict
#              Dictionary containing the selected variables.
#         """
#         selected_vars = {
#             "Tex": self.Tex[self.indices],
#             "tau": self.tau[self.indices],
#             "Aul": self.Aul[self.indices],
#             "Bul": self.Bul[self.indices],
#             "nu0_array": self.nu0_array[self.indices],
#             "Eu": self.Eu[self.indices],
#             "gu": self.gu[self.indices],
#             "glow": self.glow[self.indices],
#             "gamma": self.gamma[self.indices],
#             "Nu": self.Nu[self.indices],
#             "Nl": self.Nl[self.indices],
#             "FWHM_each_transition": self.FWHM_each_transition[self.indices],
#         }
#         return selected_vars

#     def optical_depth(self):
#         """
#         Calculate the optical depth of a transition using the different approaches.
#         In both method the change of optical depth over the line profile is not taken into account

#         Returns
#         -------
#         tau_with_correction_factor : np.array
#              Optical depth computed with the correction factor for gaussian line profile function.
#         tau_without_correction_factor : np.array
#              Optical depth computed without the correction factor for gaussian line profile function.

#         """
#         # Ensure extract_and_calculate has been called,
#         if not hasattr(self, "Tex"):  # checks if the instance variable Tex has already been set, returns True if the object already has an attribute.
#             self.extract_and_calculate()

#         conversion_factor = np.sqrt(np.pi) / (2 * np.sqrt(np.log(2)))  ## approx. 1.0645
#         # Equation used in RADEX, with correction factor for gaussian line profile function
#         tau_with_correction_factor = (
#             (constants.c**2 / (8 * np.pi * self.nu0_array**2))
#             * self.Aul
#             * ((self.Nl * self.gu / self.glow) - self.Nu)
#             / (conversion_factor * self.FWHM_each_transition)
#         )
#         # Equation used in RADEX, without correction factor for gaussian line profile function
#         tau_without_correction_factor = (
#             (constants.c**2 / (8 * np.pi * self.nu0_array**2)) * self.Aul * ((self.Nl * self.gu / self.glow) - self.Nu) / self.FWHM_each_transition
#         )

#         return tau_with_correction_factor, tau_without_correction_factor

In [ ]:
class MyClass:
    def __init__(self, cloud_name, data_of_molecule, column_density_species, width_v, Tkin):
        """
        Initialize the class with the cloud object, the data of the molecule, the column density of the species, the width of the line and the kinetic temperature.

        Parameters
        ----------
        cloud_name : pythonradex.Cloud
             The cloud object.
        data_of_molecule : dict
             The data of the molecule.
        column_density_species : float
             The column density of the species in m^2.
        width_v : float
             The width of the line in m/s
        Tkin : float
             The kinetic temperature in [K].
        """
        self.cloud_name = cloud_name
        self.data_of_molecule = data_of_molecule
        self.column_density_species = column_density_species
        self.width_v = width_v
        self.Tkin = Tkin
        self.extract_and_calculate()  # this allows optical_depth to use the extracted values from the extract_and_calculate method

    def get_transition_populations(self, names_transitions):
        """
        Relates all the transitions in names_transitions with their corresponding population densities.

        Parameters
        ----------
        names_transitions: (list of str):
             List of transitions in the format ['79-78', '14-13', ...], where each element is a string with the format "upper_level-lower_level".

        Returns
        -------
             tuple: Two arrays, one for upper and one for lower populations density.
        """
        upper_populations = []
        lower_populations = []
        for transition in names_transitions:
            upper_level, lower_level = map(int, transition.split("-"))  # split the string and convert to integers
            upper_pop = self.cloud_name.level_pop[upper_level]
            lower_pop = self.cloud_name.level_pop[lower_level]
            upper_populations.append(upper_pop)
            lower_populations.append(lower_pop)

        nu = np.array(upper_populations)
        nl = np.array(lower_populations)
        # compute upper level population density, given by: fractional population density x total column density
        Nu = nu * self.column_density_species
        Nl = nl * self.column_density_species
        return Nu, Nl

    def extract_and_calculate(self, debug=False):
        """
        Extract values from the cloud object and the data file, and calculate gamma factor

        Parameters
        ----------
        debug : boolean, optional
             If True, print debugging information. Default is False.

        Returns
        --------
        Tex: np.array
             The excitation temperature of the molecule.
        tau: np.array
             The optical depth of the transitions.
        Aul : np.array
             The Einstein coefficient for spontaneous emission [s^-1].
        Bul : np.array
             The Einstein coefficient for stimulated emission in [sr m2 Hz / Jy].
        nu0_array : np.array
             The rest frequency of the transition in Hz.
        Eu : np.array
             The energy of the upper level in [K].
        gu : np.array
             The statistical weight of the upper level.
        gamma: np.array
             Gamma factors, which is calculated using: gamma = (8 * np.pi * k * nu0**2) / (h * c**3 * Aul)
        Nu: np.array
             Upper level population density in [m-2].
        Nl: np.array
             Lower level population density in [m-2].
        FWHM_each_transition: np.array
             Frequency width of each transition in [Hz].
        """
        # Extract values
        self.Tex = self.cloud_name.Tex
        self.Aul = self.cloud_name.emitting_molecule.A21
        self.Bul = self.cloud_name.emitting_molecule.B21
        self.nu0_array = self.cloud_name.emitting_molecule.nu0
        self.tau = self.cloud_name.tau_nu(self.nu0_array)

        # levels = self.data_of_molecule["levels"]
        rad_transitions = self.data_of_molecule["radiative transitions"]

        # Extract upper energy levels in [J] and convert to Kelvin
        self.Eu = np.array([transition.up.E for transition in rad_transitions]) / constants.k

        # Get the statistical weight of the upper and lower levels
        self.gu = np.array([transition.up.g for transition in rad_transitions])
        self.glow = np.array([transition.low.g for transition in rad_transitions])

        # Get names of transitions
        names_transitions = np.array([tran.name for tran in rad_transitions])

        # Compute upper and lower level population densities
        self.Nu, self.Nl = self.get_transition_populations(names_transitions)

        # Convert velocity width to frequency width
        self.FWHM_each_transition = self.nu0_array * (self.width_v / constants.c)

        # Calculate gamma factor
        self.gamma = (8 * np.pi * constants.k * self.nu0_array**2) / (constants.h * constants.c**3 * self.Aul)

        if debug:
            # Debugging print statements
            print("Size of nu0_array:", len(self.nu0_array))
            print("Size of Eu:", len(self.Eu))
            print("E upper levels:", self.Eu)
            print("Size of gu:", len(self.gu))
            print("size exitation temperature Tex:", len(self.Tex))
            print(" gu:", self.gu)
            print("Size of glow:", len(self.glow))
            print(" glow:", self.glow)
            print("size Aul:", len(self.Aul))
            print("size FWHM_each_transition:", len(self.FWHM_each_transition))
            print("size gamma:", len(self.gamma))
            print("size Nu:", len(self.Nu))
            print("size Nl:", len(self.Nl))

        return (
            self.Tex,
            self.tau,
            self.Aul,
            self.Bul,
            self.nu0_array,
            self.Eu,
            self.gu,
            self.glow,
            self.gamma,
            self.Nu,
            self.Nl,
            self.FWHM_each_transition,
        )

    def get_selected_variables(self, k_quantum_number):
        """
        Returns the variables in extract_and_calculate which have indices selected by the given k_quantum_number.

        Parameters
        ----------
        k_quantum_number : str
             The k quantum number to filter the quantum numbers.

        Returns
        -------
        dict
             Dictionary containing the selected variables.
        """
        selected_quantum_numbers_with_indices = [
            (i, qn) for i, qn in enumerate(self.data_of_molecule["quantum numbers"]) if qn.split("_")[1] == k_quantum_number
        ]
        self.indices = [i for i, qn in selected_quantum_numbers_with_indices]
        print(self.indices)

        selected_vars = {
            "Tex": self.Tex[self.indices],
            "tau": self.tau[self.indices],
            "Aul": self.Aul[self.indices],
            "Bul": self.Bul[self.indices],
            "nu0_array": self.nu0_array[self.indices],
            "Eu": self.Eu[self.indices],
            "gu": self.gu[self.indices],
            "glow": self.glow[self.indices],
            "gamma": self.gamma[self.indices],
            "Nu": self.Nu[self.indices],
            "Nl": self.Nl[self.indices],
            "FWHM_each_transition": self.FWHM_each_transition[self.indices],
        }
        return selected_vars

    def optical_depth(self):
        """
        Calculate the optical depth of a transition using the different approaches.
        In both method the change of optical depth over the line profile is not taken into account

        Returns
        -------
        tau_with_correction_factor : np.array
             Optical depth computed with the correction factor for gaussian line profile function.
        tau_without_correction_factor : np.array
             Optical depth computed without the correction factor for gaussian line profile function.

        """
        # Ensure extract_and_calculate has been called,
        if not hasattr(self, "Tex"):  # checks if the instance variable Tex has already been set, returns True if the object already has an attribute.
            self.extract_and_calculate()

        conversion_factor = np.sqrt(np.pi) / (2 * np.sqrt(np.log(2)))  ## approx. 1.0645
        # Equation used in RADEX, with correction factor for gaussian line profile function
        tau_with_correction_factor = (
            (constants.c**2 / (8 * np.pi * self.nu0_array**2))
            * self.Aul
            * ((self.Nl * self.gu / self.glow) - self.Nu)
            / (conversion_factor * self.FWHM_each_transition)
        )
        # Equation used in RADEX, without correction factor for gaussian line profile function
        tau_without_correction_factor = (
            (constants.c**2 / (8 * np.pi * self.nu0_array**2)) * self.Aul * ((self.Nl * self.gu / self.glow) - self.Nu) / self.FWHM_each_transition
        )

        return tau_with_correction_factor, tau_without_correction_factor

In [ ]:
# extract the data from the cloud object and the data file
CH3OH_instance = MyClass(cloud, data_methanol, N, width_v, Tkin)

TexCH3OH, tauCH3OH, AulCH3OH, BulCH3OH, nu0_arrayCH3OH, EuCH3OH, gCH3OH, glow_CH3OH, gammaCH3OH, NuCH3OH, NlCH3OH, FWHMCH3OH = (
    CH3OH_instance.extract_and_calculate(debug=True)
)

# calculate the optical depth of a transition using the different approaches
tau_radex_CH3OH, tau_pythonRADEX_approx_CH3OH = CH3OH_instance.optical_depth()